In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np 
from vqls import *
from utils import *


In [2]:
func_out = {'sigmoid': sigmoid_t,'tanh': tanh_t,'elu': elu_t, 'relu': relu_t, 'sin':sin_m}
func_list = list(func_out.keys())
func_dict = {'sigmoid': .0,'tanh': 1.0,'elu':.12, 'relu':.0, 'sin':.0}

pnq = {'3':(3,8),
        '4':(4,16)}
columns=['function', 'nq', 'knot', 'training cost', 'time(s)', 'weights', 'RSS_h', 'RSS_q' ]
tot = {}
for flab, fun in func_out.items():
    for nq, knot in  pnq.values():
        file_path = f"results_{flab}_{nq}.json"
        df = pd.read_json(file_path).drop(['Condition number', 'norm(yk)', 'training_cost', 'exe_time','RSS_h'], axis =1 )
        tmp = []
        for el in df["in_train_weights"]:
             tmp.append(el[-1])
        df["weights"] = tmp
        df = df.drop(["in_train_weights"], axis = 1)
        tot[f"{flab}{nq}"] = df 

In [3]:
rss=[]
for flab, fun in func_out.items():
    for nq in [3,4]:
        dl = tot[f"{flab}{nq}"]
        rmse_scal = []
        rmse_yc = []
        for i, el in dl.iterrows():
            
            lower = 0.
            upper = 1.

            f_i = func_dict[flab]
            scaled=False    
            label = flab
            n_step = pnq[f"{nq}"][1]
            x = np.arange(lower, upper + .03, (upper-lower)/n_step).tolist() 
            xx = np.linspace(lower, upper, n_step) ##inputs sampling in the interval 0,1
            y = [fun(value,f_i) for value in xx]

            #tck=splrep(x,y,k=1) #coeffs
            norm = np.linalg.norm(y)
            y = y / norm
            matrix,vector,v_norm = GeneralizedVQS_System(n_step,label,x,xx,scaled=scaled)

            vqls_circuit = VQLS(matrix,v_norm,nq,opt='COBYLA') 

            c = np.linalg.solve(matrix,vector)

            y_c=np.dot(matrix,c) #classic

            y_fq=[]
            q = vqls_circuit.solution(el['weights']).real
            y_q=np.dot(matrix,q) #hybrid
                            #Quantum

            for a in matrix:
                y_fq.append(vqls_circuit.direct_prod2(el['weights'],a))
            rmse_yc.append(math.sqrt(np.square(np.subtract(y_c,y_fq)).mean()))
            rmse_scal.append(math.sqrt(np.square(np.subtract(y,y_fq)).mean()))
        dl["rmse_scal"] = rmse_scal
        dl["rmse_yc"] = rmse_yc


In [4]:
dk= tot["sigmoid4"]
#dk['RSS_q'] = dk['RSS_q']/dk['knot']
print(dk.pivot( columns = [], values=['rmse_yc','rmse_scal']).sort_values(["rmse_yc"]).to_markdown())

|      |   rmse_yc |   rmse_scal |
|:-----|----------:|------------:|
| (0,) |  0.30803  |   0.0776852 |
| (4,) |  0.308428 |   0.0780663 |
| (7,) |  0.309383 |   0.0781614 |
| (2,) |  0.660615 |   0.345134  |
| (1,) |  0.671942 |   0.353697  |
| (3,) |  0.936407 |   0.565645  |
| (5,) |  0.93789  |   0.567159  |
| (6,) |  0.938195 |   0.567497  |
